# GK9101 Data Science "Neuronale Netzwerke - pytorch/TensorFlow"

Andreas Sünder - 16.11.2023

## Introduction to PyTorch

### Setup

In [ ]:
import torch

### PyTorch Tensors

PyTorch arbeitet mit sogenannten Tensor-Objekten, die Arrays recht ähnlich sind. Mit `torch.zeros()` lassen sich (mehrdimensionale) Tensors mit Nullen initialisieren:

In [ ]:
z = torch.zeros(5, 3)

Mit `Tensor.dtype`, `Tensor.shape` (bzw. `Tensor.size()`) lassen sich wichtige Basisinformationen ausgeben:

In [ ]:
print(z.dtype)
print(z.shape)

Standardmäßig wird 32-bit Floating Point Precision verwendet; dies die Genauigkeit lässt sich auch über einen Parameter beim Initalisieren ändern:

In [ ]:
i = torch.ones((5, 3))
i.dtype

Diese "Gewichte" können auch zufällig initialisiert werden:

In [ ]:
torch.manual_seed(42)
torch.rand(2, 2)

Mit Tensors lassen sich auch einfache Rechenoperationen durchführen:

In [ ]:
ones = torch.ones(2, 3)
twos = torch.ones(2, 3) * 2 # Multiplikation mit Skalar

# Tensor-Addition (geht, weil gleiche Dimensionen)
threes = ones + twos
threes

Weitere mathematische Operationen:

In [ ]:
# Werte zwischen -1 und 1
r = (torch.rand(2, 2) - 0.5)
print('Werte zwischen 0 und 1:', r)

# Beträge der einzelnen Werte
print('Beträge:', r.abs())

# Trigonometrische Funktionen
print('Sinus:', r.sin())
print('Cosinus:', r.cos())
print('Tangens:', r.tan())
print('Asinus:', r.asin())

# Lineare Algebra
print('Determinate:', torch.det(r))
print('Singulärwertzerlegung:', torch.svd(r))

# Statistik
print('Standardabweichung:', torch.std_mean(r))
print('Minimum:', r.min())
print('Maximum:', r.max())

### PyTorch Modelle

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

Wir können nun unser Modell aufrufen:

In [ ]:
net = LeNet()
print(net) # Ausgabe der Struktur des Netzes

input = torch.rand(1, 1, 32, 32)
print('Input image shape:', input.shape)

output = net(input) # ruft intern forward() auf
print('Raw output:', output)
print('Output shape:', output.shape)

In [ ]:
## Trainieren eines Modells

Ziel ist es nun, ein Modell zu trainieren, welches auf Basis des CIFAR10-Datensatzes Bilder klassifizieren kann.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
  transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(train, batch_size=16, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(test, batch_size=16, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Zum Trainieren verwenden wir folgendes Modell:

In [4]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net = Net().to(device)

Für das Evaluieren des finalen Modells ist eine entsprechende Loss-Funktion notwendig (hier Cross-Entropy):

In [5]:
criterion = nn.CrossEntropyLoss()

Des Weiteren wird ein Optimizer benötigt. Der Optimizer selbst ist der Algorithmus, der die Gewichte des Modells anpasst. Hier verwenden wir Stochastic Gradient Descent (SGD):

In [6]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Zum Trainieren verwenden wir eine eigene Training-Loop:

In [7]:
num_epochs = 40
log_interval = 2000

for epoch in range(num_epochs):
  running_loss = 0.0
  
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    
    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % log_interval == log_interval - 1:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / log_interval))
      running_loss = 0.0

print('Finished training')

[1,  2000] loss: 2.224
[2,  2000] loss: 1.677
[3,  2000] loss: 1.461
[4,  2000] loss: 1.349
[5,  2000] loss: 1.254
[6,  2000] loss: 1.178
[7,  2000] loss: 1.129
[8,  2000] loss: 1.074
[9,  2000] loss: 1.023
[10,  2000] loss: 0.985
[11,  2000] loss: 0.938
[12,  2000] loss: 0.906
[13,  2000] loss: 0.874
[14,  2000] loss: 0.833
[15,  2000] loss: 0.799
[16,  2000] loss: 0.774
[17,  2000] loss: 0.752
[18,  2000] loss: 0.720
[19,  2000] loss: 0.700
[20,  2000] loss: 0.677
[21,  2000] loss: 0.645
[22,  2000] loss: 0.632
[23,  2000] loss: 0.606
[24,  2000] loss: 0.576
[25,  2000] loss: 0.569
[26,  2000] loss: 0.552
[27,  2000] loss: 0.532
[28,  2000] loss: 0.514
[29,  2000] loss: 0.498
[30,  2000] loss: 0.479
[31,  2000] loss: 0.461
[32,  2000] loss: 0.455
[33,  2000] loss: 0.432
[34,  2000] loss: 0.436
[35,  2000] loss: 0.412
[36,  2000] loss: 0.404
[37,  2000] loss: 0.376
[38,  2000] loss: 0.381
[39,  2000] loss: 0.362
[40,  2000] loss: 0.367
Finished training


Nun evaluieren wir das Modell auf dem Test-Datensatz:

In [8]:
correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %


Zum Schluss speichern wir das trainierte Modell:

In [9]:
torch.save(net.state_dict(), 'cifar_net.pt')